# Shimadzu Radspeed Pro - EXI Log Parser

This notebook takes the output of the EXI Log export from a Shimadzu X-ray system and produces a .csv file that can be appended to the Reject Analysis and Dose Metric Dashboard database.

Let's start by importing the required libraries needed to make the parser work.

In [1]:
import pandas as pd
import os

from ipywidgets import *
from tkinter import Tk, filedialog
from IPython.display import clear_output, display

The template for the database used in the Reject Analysis and Dose Metric Dashboard is as follows:

In [2]:
cols_list_final = ['Asset Number','Manufacturer','Model','Image Date','Image Time','Body Part','View','Exposure Index','KAP (uGy.m2)','kVp','Exposure (mAs)','Exposure time (ms)','Image Status','Reject Reason']

df_template = pd.DataFrame(columns=[*cols_list_final])
df_template

Empty DataFrame
Columns: [Asset Number, Manufacturer, Model, Image Date, Image Time, Body Part, View, Exposure Index, KAP (uGy.m2), kVp, Exposure (mAs), Exposure time (ms), Image Status, Reject Reason]
Index: []

## Selecting file from file browser

To clean up an export, you'll need to select the exported files from the Philips QA Tool using the "File Select" button below.
<br>
You will also be asked for your asset number so that we can group systems in the Dashboard by facility.

In [3]:
# Select input log from file location

def select_files(b):
    clear_output()
    root = Tk()
    root.withdraw() # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True) # Raise the root to the top of all windows.
    b.files = filedialog.askopenfilename() # List of selected files will be set button's file attribute.
    print(b.files) # Print the list of files selected.

fileselect = Button(description="File select")
fileselect.on_click(select_files)

display(fileselect)

C:/Users/bernardm/GitHub/JupyterNotebooks/rejectAnalysis/inputdata/Radspeed Pro example log.xlsx


## Adding Asset Number

In the next section, the file that you've selected will be printed out and you'll be asked to input the Asset Number of the system. Type the Asset Number in the box provided and then press "ENTER".

In [4]:
files = fileselect.files
print("The file you've selected is: ", files)
print()
print("What is the asset number of the system?")

AssetNumber = input()


The file you've selected is:  C:/Users/bernardm/GitHub/JupyterNotebooks/rejectAnalysis/inputdata/Radspeed Pro example log.xlsx

What is the asset number of the system?
1234567789


## Clean up dataframe

Let's do some clean-up on the .csv file:
- Define the separator. By default, this will be ";". However, this can be set to something different during export (| or , or -). Change the code below if your separator is something other than ";".

In [5]:
df = pd.read_excel(files)
df

Content date Content time  Accession number Exposure mode  \
0    2021-01-13     18:56:11               NaN           Rad   
1    2021-03-11     18:52:27               NaN           Rad   
2    2021-03-11     18:55:52               NaN           Rad   
3    2021-05-24     10:15:36               NaN           Rad   
4    2021-05-24     11:12:28               NaN           Rad   
5    2021-05-24     11:13:34               NaN           Rad   
6    2021-05-24     11:14:27               NaN           Rad   
7    2021-05-24     11:15:51               NaN           Rad   
8    2021-06-06     18:57:29               NaN           Rad   
9    2021-06-06     18:57:58               NaN           Rad   
10   2021-06-06     18:58:28               NaN           Rad   
11   2021-10-26     13:19:28               NaN           Rad   
12   2021-10-26     13:20:06               NaN           Rad   
13   2021-10-26     13:27:23               NaN           Rad   
14   2021-10-26     13:28:05               NaN           Rad   
15   2021-10-26     13:29:43               NaN           Rad   
16   2021-10-26     13:30:29               NaN           Rad   
17   2021-10-26     13:53:12               NaN           Rad   
18   2021-10-26     13:54:41               NaN           Rad   
19   2021-10-26     13:55:24               NaN           Rad   
20   2021-10-26     13:57:06               NaN           Rad   
21   2021-10-26     13:58:10               NaN           Rad   
22   2021-10-26     14:04:51               NaN           Rad   
23   2021-10-26     14:05:21               NaN           Rad   
24   2021-10-26     14:07:05               NaN           Rad   
25   2021-10-26     14:08:52               NaN           Rad   
26   2021-10-26     14:09:44               NaN           Rad   

                                    Protocol Bodypart examined  \
0                             Chest (0-3) AP             CHEST   
1         Chest AP Scatter Correction 100kVp             CHEST   
2        AP Abdomen Scatter Correction 80kVp           ABDOMEN   
3                                   Chest PA             CHEST   
4         Chest AP Scatter Correction 100kVp             CHEST   
5        Humerus AP Scatter Correction 60kVp           HUMERUS   
6   Humerus Lateral Scatter Correction 60kVp           HUMERUS   
7                              Elbow Lateral             ELBOW   
8                                     Finger               ARM   
9                                     Finger               ARM   
10                                    Finger               ARM   
11                         Abdomen AP Supine           ABDOMEN   
12                         Abdomen AP Supine           ABDOMEN   
13                                  Chest PA             CHEST   
14                             Chest Lateral             CHEST   
15                              Knee Lateral              KNEE   
16                                   Knee AP              KNEE   
17                                  Chest PA             CHEST   
18                                 Pelvis AP            PELVIS   
19                                 Pelvis AP            PELVIS   
20                     Lumbar Spine AP Erect             SPINE   
21                Lumbar Spine Lateral Erect            LSPINE   
22                              Knee Lateral              KNEE   
23                              Knee Lateral              KNEE   
24                              Lower Leg AP         EXTREMITY   
25                         Lower Leg Lateral         EXTREMITY   
26                         Lower Leg Lateral         EXTREMITY   

   Patient orientation View position Operator name Reject date  ...  EIt  \
0                  L\F           NaN  Radiographer         NaT  ...  100   
1                  L\F           NaN  Radiographer         NaT  ...  300   
2                  L\F           NaN  Radiographer         NaT  ...  225   
3                  L\F            PA  Radiographer       

## Adding unique identifiers

If the .csv file has been read in correctly above, you should see a table with all the values from the log. 
<br>
Let's add a few identifiers for filtering purposes:
- Asset Number
- Manufacturer
- Model


In [45]:
cols_list = ['Asset Number','Manufacturer','Model','Image Status']
df = df.reindex(columns=[*cols_list,*df.columns.tolist()])

df['Asset Number'] = AssetNumber
df['Manufacturer'] = "Shimadzu"
df['Model'] = "Radspeed Pro"
df['Image Status'] = df['Reject date'].apply(lambda x: 'Rejected' if pd.notnull(x) else 'Approved')

df

Asset Number Manufacturer         Model Image Status Content date  \
0     123456789     Shimadzu  Radspeed Pro     Approved   2021-01-13   
1     123456789     Shimadzu  Radspeed Pro     Approved   2021-03-11   
2     123456789     Shimadzu  Radspeed Pro     Approved   2021-03-11   
3     123456789     Shimadzu  Radspeed Pro     Approved   2021-05-24   
4     123456789     Shimadzu  Radspeed Pro     Approved   2021-05-24   
5     123456789     Shimadzu  Radspeed Pro     Approved   2021-05-24   
6     123456789     Shimadzu  Radspeed Pro     Approved   2021-05-24   
7     123456789     Shimadzu  Radspeed Pro     Approved   2021-05-24   
8     123456789     Shimadzu  Radspeed Pro     Approved   2021-06-06   
9     123456789     Shimadzu  Radspeed Pro     Approved   2021-06-06   
10    123456789     Shimadzu  Radspeed Pro     Approved   2021-06-06   
11    123456789     Shimadzu  Radspeed Pro     Approved   2021-10-26   
12    123456789     Shimadzu  Radspeed Pro     Approved   2021-10-26   
13    123456789     Shimadzu  Radspeed Pro     Approved   2021-10-26   
14    123456789     Shimadzu  Radspeed Pro     Approved   2021-10-26   
15    123456789     Shimadzu  Radspeed Pro     Approved   2021-10-26   
16    123456789     Shimadzu  Radspeed Pro     Approved   2021-10-26   
17    123456789     Shimadzu  Radspeed Pro     Approved   2021-10-26   
18    123456789     Shimadzu  Radspeed Pro     Rejected   2021-10-26   
19    123456789     Shimadzu  Radspeed Pro     Approved   2021-10-26   
20    123456789     Shimadzu  Radspeed Pro     Approved   2021-10-26   
21    123456789     Shimadzu  Radspeed Pro     Approved   2021-10-26   
22    123456789     Shimadzu  Radspeed Pro     Rejected   2021-10-26   
23    123456789     Shimadzu  Radspeed Pro     Approved   2021-10-26   
24    123456789     Shimadzu  Radspeed Pro     Approved   2021-10-26   
25    123456789     Shimadzu  Radspeed Pro     Approved   2021-10-26   
26    123456789     Shimadzu  Radspeed Pro     Approved   2021-10-26   

   Content time  Accession number Exposure mode  \
0      18:56:11               NaN           Rad   
1      18:52:27               NaN           Rad   
2      18:55:52               NaN           Rad   
3      10:15:36               NaN           Rad   
4      11:12:28               NaN           Rad   
5      11:13:34               NaN           Rad   
6      11:14:27               NaN           Rad   
7      11:15:51               NaN           Rad   
8      18:57:29               NaN           Rad   
9      18:57:58               NaN           Rad   
10     18:58:28               NaN           Rad   
11     13:19:28               NaN           Rad   
12     13:20:06               NaN           Rad   
13     13:27:23               NaN           Rad   
14     13:28:05               NaN           Rad   
15     13:29:43               NaN           Rad   
16     13:30:29               NaN           Rad   
17     13:53:12               NaN           Rad   
18     13:54:41               NaN           Rad   
19     13:55:24               NaN           Rad   
20     13:57:06               NaN           Rad   
21     13:58:10               NaN           Rad   
22     14:04:51               NaN           Rad   
23     14:05:21               NaN           Rad   
24     14:07:05               NaN           Rad   
25     14:08:52               NaN           Rad   
26     14:09:44               NaN           Rad   

                                    Protocol Bodypart examined  ...  EIt  \
0                             Chest (0-3) AP             CHEST  ...  100   
1         Chest AP Scatter Correction 100kVp             CHEST  ...  300   
2        AP Abdomen Scatter Correction 80kVp           ABDOMEN  ...  225   
3                                   Chest PA             CHEST  ...  100   
4         Chest AP Scatter Correction 100kVp             CHEST  ...  300   
5        Humerus AP Scatter Correction 60kVp           HUMERUS  ...  225   
6   Humerus Lateral Scatt

## Putting data into the template dataframe

Let's match the .csv columns into the template:
- rename the columns of the original .csv file to match the template
- remove any columns we don't need
- rearrange the columns to match the template

In [46]:
df = df.rename(columns={"Bodypart examined": "Body Part",
                       "View position": "View",
                        "EI": "Exposure Index",
                        "kV": "kVp",
                        "mAs": "Exposure (mAs)",
                        "ms": "Exposure time (ms)",
                        "Reject reason": "Reject Reason",
                        "DAP": "KAP (uGy.m2)",
                        "Content date": "Image Date",
                        "Content time": 'Image Time'
                       })

df_out = df[[*cols_list_final]]
df_out

Asset Number Manufacturer         Model Image Date Image Time  Body Part  \
0     123456789     Shimadzu  Radspeed Pro 2021-01-13   18:56:11      CHEST   
1     123456789     Shimadzu  Radspeed Pro 2021-03-11   18:52:27      CHEST   
2     123456789     Shimadzu  Radspeed Pro 2021-03-11   18:55:52    ABDOMEN   
3     123456789     Shimadzu  Radspeed Pro 2021-05-24   10:15:36      CHEST   
4     123456789     Shimadzu  Radspeed Pro 2021-05-24   11:12:28      CHEST   
5     123456789     Shimadzu  Radspeed Pro 2021-05-24   11:13:34    HUMERUS   
6     123456789     Shimadzu  Radspeed Pro 2021-05-24   11:14:27    HUMERUS   
7     123456789     Shimadzu  Radspeed Pro 2021-05-24   11:15:51      ELBOW   
8     123456789     Shimadzu  Radspeed Pro 2021-06-06   18:57:29        ARM   
9     123456789     Shimadzu  Radspeed Pro 2021-06-06   18:57:58        ARM   
10    123456789     Shimadzu  Radspeed Pro 2021-06-06   18:58:28        ARM   
11    123456789     Shimadzu  Radspeed Pro 2021-10-26   13:19:28    ABDOMEN   
12    123456789     Shimadzu  Radspeed Pro 2021-10-26   13:20:06    ABDOMEN   
13    123456789     Shimadzu  Radspeed Pro 2021-10-26   13:27:23      CHEST   
14    123456789     Shimadzu  Radspeed Pro 2021-10-26   13:28:05      CHEST   
15    123456789     Shimadzu  Radspeed Pro 2021-10-26   13:29:43       KNEE   
16    123456789     Shimadzu  Radspeed Pro 2021-10-26   13:30:29       KNEE   
17    123456789     Shimadzu  Radspeed Pro 2021-10-26   13:53:12      CHEST   
18    123456789     Shimadzu  Radspeed Pro 2021-10-26   13:54:41     PELVIS   
19    123456789     Shimadzu  Radspeed Pro 2021-10-26   13:55:24     PELVIS   
20    123456789     Shimadzu  Radspeed Pro 2021-10-26   13:57:06      SPINE   
21    123456789     Shimadzu  Radspeed Pro 2021-10-26   13:58:10     LSPINE   
22    123456789     Shimadzu  Radspeed Pro 2021-10-26   14:04:51       KNEE   
23    123456789     Shimadzu  Radspeed Pro 2021-10-26   14:05:21       KNEE   
24    123456789     Shimadzu  Radspeed Pro 2021-10-26   14:07:05  EXTREMITY   
25    123456789     Shimadzu  Radspeed Pro 2021-10-26   14:08:52  EXTREMITY   
26    123456789     Shimadzu  Radspeed Pro 2021-10-26   14:09:44  EXTREMITY   

   View  Exposure Index  KAP (uGy.m2)  kVp  Exposure (mAs)  \
0   NaN       98.198861            28   60            1.60   
1   NaN      447.475712           228  100            3.20   
2   NaN      714.906577          1563   80           12.80   
3    PA       69.618978           121  100            3.50   
4   NaN      169.705380            84  100            3.20   
5   NaN      171.089818            99   60            6.40   
6   NaN       81.681921           159   60           10.24   
7   NaN      372.210790            25   63            1.92   
8   NaN      150.328283             1   50            1.60   
9   NaN      109.328940             1   50            1.60   
10  NaN       99.135756             1   50            1.60   
11   AP      281.603293          1555   80           17.60   
12   AP      358.358704          1831   80           25.60   
13   PA       57.440564           202  100            3.25   
14  NaN      147.411088          1219  110           18.00   
15  NaN      113.355919            48   60            2.00   
16  NaN      185.570818            55   60            2.00   
17   PA      119.664164           264  100            3.50   
18  NaN      570.980557          8066   80           56.64   
19  NaN      575.638545          8035   80           56.32   
20  NaN      337.630713          3724   85           42.56   
21  NaN      174.362648          4670   90          102.40   
22  NaN      118.375860            42   60            1.92   
23  NaN      120.315131            42   60            2.00   
24  NaN      136.647250             7   70            0.64   
25  NaN      304.615043            22   66            0.64   
26  NaN      224.306862            37   66            0.96   

    Exposure time (ms) Image Status      Reject Reaso

## Output of dataframe into .csv file

Check the output table above and confirm that the information matches the column names. 
If everything is OK, let's export the cleaned up .csv file into an output file. 

A .csv file will be created with the Asset Number and a datestamp.

In [47]:
# Add timestamp to filename
from datetime import datetime
date = datetime.now().strftime("%Y_%m_%d_%I_%S%p")

df_out.to_csv((r'C:\Users\BernardM\GitHub\JupyterNotebooks\rejectAnalysis\outputdata\\'
               +str(AssetNumber)
               +'_'
               +str(date)
               +'.csv')
               ,index = False, header = True)

print("The output file has been successfully created.")

The output file has been successfully created.


This output file can now be appended to the Reject Analysis and Dose Metric Dashboard database.